In [2]:
#this is a basic test run for getting what will eventually be in my StreamLit app to work before I put it in there

!pip install ultralytics
!pip install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

In [3]:
import os
import cv2
import torch
import easyocr
import numpy as np
from tensorflow.keras.models import load_model
from ultralytics import YOLO
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:

yolo_model = YOLO('yolov8n.pt')

100%|██████████| 6.25M/6.25M [00:00<00:00, 50.0MB/s]


In [7]:
celtic_model_path = '/content/drive/My Drive/GA/Capstone/capstone/models/celtics_classifier_fine_tuned.h5'
celtic_model = load_model(celtic_model_path)

In [8]:
reader = easyocr.Reader(['en'])

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [9]:
confidence_threshold = 0.5  # confidence threshold for person detection
center_weight = 0.5  # how heavily I weight a person's proximity to the center
sharpness_weight = 0.5  # how heavily I weight the sharpness of the identified person

In [10]:
def calculate_sharpness(roi):
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

In [12]:
def process_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image {image_path}. Skipping.")
        return

    # acquiring and storing the image dimensions
    height, width = image.shape[:2]
    image_center = (width / 2, height / 2)

    # basic yolo inference
    results = yolo_model(image)

    if len(results[0].boxes) > 0:
        # persons are feature 0 in the image detection category
        person_boxes = [box for box in results[0].boxes if box.cls == 0 and box.conf >= confidence_threshold]

        if len(person_boxes) > 0:
            # sorting boxes based on adjusted score
            scored_boxes = []
            for box in person_boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                box_center = ((x1 + x2) / 2, (y1 + y2) / 2)

                # calculating distance from center (basic geometry)
                distance = np.sqrt((box_center[0] - image_center[0]) ** 2 + (box_center[1] - image_center[1]) ** 2)

                # defining the ROI
                roi = image[y1:y2, x1:x2]

                # calculating sharpness is easy
                sharpness = calculate_sharpness(roi)

                # adjusting the confidence score based on these weights
                adjusted_score = box.conf - center_weight * (distance / max(width, height)) + sharpness_weight * sharpness

                scored_boxes.append((adjusted_score, x1, y1, x2, y2, roi))

            # sorting by score and taking the top 3 boxes
            scored_boxes = sorted(scored_boxes, key=lambda x: x[0], reverse=True)[:3]

            celtic_detected = False

            for score, x1, y1, x2, y2, roi in scored_boxes:
                # resizing the ROI for MobileNet Preprocessing
                roi_resized = cv2.resize(roi, (224, 224))

                # actual preprocessing for MobileNet
                roi_resized = roi_resized / 255.0
                roi_resized = np.expand_dims(roi_resized, axis=0)

                # predicting using MobileNet
                celtic_prediction = celtic_model.predict(roi_resized)
                is_celtic = np.argmax(celtic_prediction) == 1

                if is_celtic:
                    celtic_detected = True
                    print("Celtic player detected!")

                    # bringing in EasyOCR to detect text (jersey number)
                    ocr_results = reader.readtext(roi)

                    jersey_number = ''.join([res[1] for res in ocr_results if res[1].isdigit()])

                    if jersey_number:
                        print(f"Jersey number detected: {jersey_number}")
                    else:
                        print("Celtic identified, digits unknown.")
                else:
                    print("Non-Celtic player detected.")

            if not celtic_detected:
                print("No Celtics Identified.")
        else:
            print("No high-confidence players detected.")
    else:
        print("No players detected.")

In [34]:
# testing this on various images from my dataset (this was a crappy model so I went back to an older one that was much better)

image_path = '/content/drive/My Drive/GA/Capstone/capstone/images/CelticsImages/Solo/200.png'
process_image(image_path)



0: 640x512 5 persons, 1 frisbee, 1 sports ball, 167.3ms
Speed: 4.4ms preprocess, 167.3ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 512)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Non-Celtic player detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Non-Celtic player detected.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Non-Celtic player detected.
No Celtics Identified.
